In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5Model
# Load the T5 tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5Model.from_pretrained(model_name)
# Load the Excel file
excel_file_path = 's22.xlsx'  # Replace with your file path
df = pd.read_excel(excel_file_path)
# Assuming the answers are in a column named 'answers'
answers = df['input'].tolist()  # Adjust the column name as needed
# Function to generate embeddings
def generate_embeddings(texts):
    # Ensure all inputs are strings
    texts = [str(text) for text in texts]
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs, decoder_input_ids=inputs.input_ids)
        # Get the last hidden states
        embeddings = outputs.last_hidden_state
    # Mean embedding for each text
    mean_embeddings = embeddings.mean(dim=1)
    return mean_embeddings
# Process in batches to avoid memory issues
batch_size = 16
all_embeddings = []
for i in range(0, len(answers), batch_size):
    batch_answers = answers[i:i + batch_size]
    embeddings = generate_embeddings(batch_answers)
    all_embeddings.append(embeddings)
# Concatenate all embeddings into a single tensor
all_embeddings = torch.cat(all_embeddings, dim=0)
# Convert to a DataFrame for easier saving
embeddings_df = pd.DataFrame(all_embeddings.numpy())
# Save embeddings to a new Excel file
embeddings_df.to_excel('output2.xlsx', index=False)  # Adjust the save path as needed
print("Embeddings generated and saved successfully.")